In [768]:
import pandas as pd
import pickle
import os
import datetime as dt
from alpha_vantage.timeseries import TimeSeries

In [773]:
def getStoredData(srtdt, enddt, ticker):
    #currently assumes that csv data is organised in format: Date,Open,High,Low,Close,Adj Close,Volume
    #also assumes that the name of the csv is the same as that as the ticker
    path = r'C:\Users\Edward Stables\Documents\Programming\Jupyter\Man AHL\Data\Initial Datasets'
    beginning_of_time='2000-01-01'#sets the earliest date required to be stored
    os.chdir(path) #sets the directory for the files
    
    try:
        #attempt to load the csv file from the path directory
        frame = pd.read_csv(ticker+'.csv')
        #set 'date' as index
        frame = frame.set_index(['date'])
        
        if dt.datetime.strptime(frame.index[-1], '%Y-%m-%d').date() < dt.datetime.today().date():       
            #if the data is not up-to-date:
            updateData = getData(dt.datetime.strptime(frame.index[-1], '%Y-%m-%d').date().strftime('%Y-%m-%d'), dt.datetime.today().date().strftime('%Y-%m-%d'), ticker)
            #appends the new data to the imported data
            frame = frame.append(updateData[1:])
            #saves the data back to storage
            frame.to_csv(ticker+'.csv')
            
    except FileNotFoundError:
        #if the file doesn't exist then send a request to get the data for the ticker's dates#
        with open(ticker+'.csv', "w"):
            pass
        #gets the new data, from 1/1/2000 to the current day
        frame = getData(beginning_of_time, dt.datetime.today().date().strftime('%Y-%m-%d'), ticker)
        #saves it to disk
        frame.to_csv(ticker+'.csv')
        #reads the requested value from disk
        frame = pd.read_csv(ticker+'.csv')
        #sets date as index
        frame = frame.set_index(['date'])
    #returns the selected values
    return frame[(frame.index >= srtdt) & (frame.index <= enddt)]


In [774]:
def getData(strdt, enddt, tic):
    API_KEY = '4U0DSJC208E4D8R7'
    #makes api call
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=tic, outputsize='full')
    #selects required data from full datarange returned
    dataRange = data.loc[strdt: enddt]
    return dataRange

In [775]:
print(getStoredData('2000-01-01', '2018-04-20', 'MSFT'))

            1. open  2. high   3. low  4. close   5. volume
date                                                       
2000-01-03  117.370  118.620  112.000    116.56  26614200.0
2000-01-04  113.560  117.120  112.250    112.62  27059500.0
2000-01-05  111.120  116.370  109.370    113.81  32029800.0
2000-01-06  112.190  113.870  108.370    110.00  27488300.0
2000-01-07  108.620  112.250  107.310    111.44  31006800.0
2000-01-10  113.440  113.690  111.370    112.25  22481800.0
2000-01-11  111.500  114.250  108.690    109.37  23371800.0
2000-01-12  108.500  108.870  104.440    105.81  33266200.0
2000-01-13  104.370  108.620  101.500    107.81  41572000.0
2000-01-14  107.190  113.940  105.750    112.25  36708200.0
2000-01-18  111.810  116.500  111.750    115.31  40741800.0
2000-01-19  110.500  111.500  106.000    107.00  48784100.0
2000-01-20  107.060  109.690  105.870    106.00  28174900.0
2000-01-21  107.000  107.250  103.250    103.75  34208100.0
2000-01-24  103.800  105.690  100.810   